In [2]:
import matplotlib.pyplot as plt
import numpy as np
from astropy.io import fits
import pyke
from pyke import KeplerTargetPixelFile
from glob import glob
from scipy.signal import convolve2d
import everest
import os
import fnmatch
from ipywidgets import interact
from ipywidgets import interact, interactive, fixed, interact_manual
from scipy.signal import convolve2d
from astropy.wcs import WCS
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

## Look for transients frame by frame

In [3]:
pixelfile = '/Volumes/TOSHIBA EXT/K2/c5/200000000/08000/ktwo200008650-c05_lpd-targ.fits.gz'
#pixelfile = '../data/ktwo212535880-c06_lpd-targ.fits'
save = '/Users/ryanr/Documents/PhD/coding/Kepler/K2/c5/'

hdu = fits.open(pixelfile)
dat = hdu[1].data
datacube = dat["FLUX"]
time = dat["TIME"]


n_steps = 12
std_vec = np.zeros(n_steps)
for i in range(n_steps):
    std_vec[i] = np.nanstd(datacube[i:-n_steps+i:n_steps,:,:] - datacube[i+n_steps*80,:,:])
Framemin = np.where(std_vec==np.nanmin(std_vec))[0][0]
# Make a mask of the target object, using the reference frame 
Negmask = datacube>0
datacube = datacube*Negmask
Mask = datacube[Framemin,:,:]/(np.nanmedian(datacube[Framemin,:,:])+2*np.nanstd(datacube[Framemin,:,:]))
Mask[Mask>=1] = np.nan
#Mask[0<Mask] = np.nan
Mask[Mask<1] = 1

Maskv2 = datacube[Framemin,:,:]*Mask/(np.nanmedian(datacube[Framemin,:,:]*Mask)+2*np.nanstd(datacube[Framemin,:,:]*Mask))
Maskv2[Maskv2>=1] = np.nan
#Maskv2[0<Maskv2] = np.nan
Maskv2[Maskv2<1] = 1

FileNotFoundError: [Errno 2] No such file or directory: '/Volumes/TOSHIBA EXT/K2/c5/200000000/08000/ktwo200008650-c05_lpd-targ.fits.gz'

In [ ]:
Maskdata = datacube*Maskv2#-datacube[Framemin]*Maskv2
Negmask = Maskdata>0
Maskdata = Maskdata*Negmask
#Maskdata = Maskdata[Framemin::12,:,:]



In [ ]:
hdu = fits.open(pixelfile)
dat = hdu[1].data
datacube = dat["FLUX"]
time = dat["TIME"]


n_steps = 12
std_vec = np.zeros(n_steps)
for i in range(n_steps):
    std_vec[i] = np.nanstd(datacube[i:-n_steps+i:n_steps,:,:] - datacube[i+n_steps*80,:,:])
Framemin = np.where(std_vec==np.nanmin(std_vec))[0][0]
# Make a mask of the target object, using the reference frame 
Negmask = datacube>0
datacube = datacube*Negmask
Mask = datacube[Framemin,:,:]/(np.nanmedian(datacube[Framemin,:,:])+2*np.nanstd(datacube[Framemin,:,:]))
Mask[Mask>=1] = np.nan
#Mask[0<Mask] = np.nan
Mask[Mask<1] = 1

Maskv2 = datacube[Framemin,:,:]*Mask/(np.nanmedian(datacube[Framemin,:,:]*Mask)+2*np.nanstd(datacube[Framemin,:,:]*Mask))
Maskv2[Maskv2>=1] = np.nan
#Maskv2[0<Maskv2] = np.nan
Maskv2[Maskv2<1] = 1

Maskdata = datacube*Maskv2
Negmask = Maskdata>0
Maskdata = Maskdata*Negmask
Maskdata = Maskdata[Framemin::12,:,:]

Stdframe = np.ones(Maskdata.shape)
framemask = np.ones(Maskdata.shape)
framemask[:,:,:] = False
stddist = np.zeros(Maskdata.shape[0])
for i in range(Maskdata.shape[0]):
    stddist[i] = np.nanstd(Maskdata[i,:,:])


for i in range(Stdframe.shape[0]+1):
    if i < Stdframe.shape[0]:
        if np.nanstd(Maskdata[i,:,:]) > np.nanmedian(stddist)+np.nanstd(stddist):
            Stdframe[i,:,:] = Maskdata[i,:,:]/(np.nanmedian(Maskdata[i,:,:])+2*np.nanstd(Maskdata[i,:,:]))
            framemask[i,:,:] = Stdframe[i,:,:] >= 1
        else:
            framemask[i,:,:] = 0

length = 0
i = 0

while length < 3 and i < framemask.shape[0]:
    Frames = np.ones(Maskdata[Framemin].shape)
    length = 0
    while Frames.any() and i+1 < framemask.shape[0]:
        length+=1
        Frames = Frames*framemask[i,:,:]*framemask[i+1,:,:]
        if length > 3 and ~Frames.any():
            tranmask.append(Frames)
        Previous = Frames 
        i+=1

for j in range(len(tranmask)):
    # Generate a light curve from the transient masks
    LC = np.nansum(datacube*tranmask[j], axis = 0)
    fig = plt.figure(figsize=(20,8))
    f, (ax0, ax1) = plt.subplots(1,2, gridspec_kw = {'width_ratios':[3, 1]})
    ax0.plot(time, LC,'.')
    ax1.imshow(tranmask[j])
    plt.suptitle(pixelfile.split('/')[-1].split('-')[0])
    ax0.xlabel('Time')
    ax0.ylabel('Flux')
    plt.tight_layout()
    plt.savefig(save+pixelfile.split('/')[-1].split('-')[0]+'.pdf');           


In [ ]:
length = 0
i = 0

while length < 3 and i < Stdframe.shape[0]:
    Frames = np.ones(Maskdata[Framemin].shape)
    length = 0
    while Frames.any() and i+1 < Stdframe.shape[0]:
        length+=1
        Frames = Frames*framemask[i,:,:]*framemask[i+1,:,:]
        if length > 3 and ~Frames.any():
            tranmask.append(Frames)
        Previous = Frames 
        i+=1

for j in range(len(tranmask)):
    # Generate a light curve from the transient masks
    LC = np.nansum(datacube*tranmask[j], axis = 0)
    fig = plt.figure(figsize=(20,8))
    f, (ax0, ax1) = plt.subplots(1,2, gridspec_kw = {'width_ratios':[3, 1]})
    ax0.plot(time, LC,'.')
    ax1.imshow(tranmask[j])
    plt.suptitle(pixelfile.split('/')[-1].split('-')[0])
    ax0.xlabel('Time')
    ax0.ylabel('Flux')
    plt.tight_layout()
    plt.savefig(save+pixelfile.split('/')[-1].split('-')[0]+'.pdf');

Function to search the pixel files for possible transients

In [ ]:
def K2tranPix(pixelfile,save):
    # Loading in data
    hdu = fits.open(pixelfile)
    dat = hdu[1].data
    datacube = dat["FLUX"]
    time = dat["TIME"]

    # Finding the reference frame
    n_steps = 12
    std_vec = np.zeros(n_steps)
    for i in range(n_steps):
        std_vec[i] = np.nanstd(datacube[i:-n_steps+i:n_steps,:,:] - datacube[i+n_steps*80,:,:])
    Framemin = np.where(std_vec==np.nanmin(std_vec))[0][0]
    
    # Remove all negative flux values, not physical
    datacube[datacube < 0] = np.nan
    
    # Make a mask of the target object, using the reference frame 
    Mask = datacube[Framemin,:,:]/(np.nanmedian(datacube[Framemin,:,:])+2*np.nanstd(datacube[Framemin,:,:]))
    Mask[Mask>=1] = np.nan
    Mask[Mask<1] = 1
    # Generate a second mask from remainder of the first. This grabs the fainter pixels around known sources
    Maskv2 = datacube[Framemin,:,:]*Mask/(np.nanmedian(datacube[Framemin,:,:]*Mask)+2*np.nanstd(datacube[Framemin,:,:]*Mask))
    Maskv2[Maskv2>=1] = np.nan
    Maskv2[Maskv2<1] = 1
    # Apply mask to data
    Maskdata = datacube*Maskv2
    #Maskdata[Maskdata < 0] = np.nan
    #Maskdata = Maskdata[Framemin::12,:,:]
    # Calculating the standard deviations of all frames, from which the significance of each
    # frame's std can be compared.
    stddist = np.zeros(Maskdata.shape[0])
    for i in range(Maskdata.shape[0]):
        stddist[i] = np.nanstd(Maskdata[i,:,:])
    
    # Section to calculate if any points are significant in a frame
    Stdframe = np.ones(Maskdata.shape)
    framemask = np.ones(Maskdata.shape)
    framemask[:,:,:] = False
    
    for i in range(Stdframe.shape[0]+1):
        if i < Stdframe.shape[0]:
            if np.nanstd(Maskdata[i,:,:]) > np.nanmedian(stddist)+2*np.nanstd(stddist): # Conditioning on std for noise
                Stdframe[i,:,:] = Maskdata[i,:,:]/(np.nanmedian(Maskdata[i,:,:])+2*np.nanstd(Maskdata[i,:,:]))
                framemask[i,:,:] = Stdframe[i,:,:] >= 1
            else:
                framemask[i,:,:] = 0
    
    
    # Determining if there are any points that could be transients.
    length = 0
    i = 0
    tranmask=[]
    while length < 3 and i < framemask.shape[0]:
        Frames = np.ones(Maskdata[Framemin].shape)
        length = 0
        while Frames.any() and i+1 < framemask.shape[0]:
            #print(i)
            length+=1
            Frames = Frames*framemask[i,:,:]*framemask[i+1,:,:]
            if length > 3 and ~Frames.any():
                tranmask.append(Previous)
                #print(str(length))
            Previous = Frames 
            i+=1
    # Plotting
    #print(len(tranmask))
    if len(tranmask)>0:
        for j in range(len(tranmask)):
            # Generate a light curve from the transient masks
            LC = np.nansum(datacube*tranmask[j], axis = (1,2))
            fig = plt.figure()#figsize=(16,6))
            f, (ax0, ax1) = plt.subplots(1,2, gridspec_kw = {'width_ratios':[3, 1]})
            ax0.plot(time, LC,'.')
            
            ax1.imshow(tranmask[j], origin='lower')
            plt.title('Transient Mask')
            ax0.set_title(pixelfile.split('/')[-1].split('-')[0])
            ax0.set_xlabel('Time')
            ax0.set_ylabel('Flux')
            plt.tight_layout()
            plt.savefig(save+pixelfile.split('/')[-1].split('-')[0]+' '+str(j)+'.pdf', bbox_inches = 'tight'); 
        Result = 'Possible'
    else: 
        Result = 'No'
    return print(Result)

In [ ]:
path = '/Volumes/TOSHIBA EXt/K2/c5/*/*/'
save = '/Users/ryanr/Documents/PhD/coding/Kepler/K2/c5/K2tranPix/'

for filename in glob(path+'*.gz'):
    K2tranPix(filename,save)

In [18]:
def K2tranPix(pixelfile,save): # More efficient in checking frames
    # Loading in data
    hdu = fits.open(pixelfile)
    dat = hdu[1].data
    datacube = dat["FLUX"]
    time = dat["TIME"]

    # Finding the reference frame
    n_steps = 12
    std_vec = np.zeros(n_steps)
    for i in range(n_steps):
        std_vec[i] = np.nanstd(datacube[i:-n_steps+i:n_steps,:,:] - datacube[i+n_steps*80,:,:])
    Framemin = np.where(std_vec==np.nanmin(std_vec))[0][0]
    
    # Remove all negative flux values, not physical
    #datacube[datacube < 0] = np.nan
    
    # Make a mask of the target object, using the reference frame 
    Mask = datacube[Framemin,:,:]/(np.nanmedian(datacube[Framemin,:,:])+np.nanstd(datacube[Framemin,:,:]))
    Mask[Mask>=1] = np.nan
    Mask[Mask<1] = 1
    # Generate a second mask from remainder of the first. This grabs the fainter pixels around known sources
    Maskv2 = datacube[Framemin,:,:]*Mask/(np.nanmedian(datacube[Framemin,:,:]*Mask)+np.nanstd(datacube[Framemin,:,:]*Mask))
    Maskv2[Maskv2>=1] = np.nan
    Maskv2[Maskv2<1] = 1
    # Apply mask to data
    Maskdata = datacube*Maskv2
    Maskdata[Maskdata < 0] = 0
    #Maskdata = Maskdata[Framemin::12,:,:]
    
    kernel = np.ones((3,3))
    temp = np.ma.masked_invalid(Maskv2)
    Conv = convolve2d(temp.mask, kernel, mode="same")
    if Mask.shape[0]>1 and Mask.shape[1]>1:
        MotionControl = Conv < 2
    else:
        MotionControl = Conv < 1
    Maskv2 = Maskv2*MotionControl
    
    # Calculating the standard deviations of all frames, from which the significance of each
    # frame's std can be compared.
    stddist = np.zeros(Maskdata.shape[0])
    for i in range(Maskdata.shape[0]):
        stddist[i] = np.nanstd(Maskdata[i,:,:])
    
    # Section to calculate if any points are significant in a frame
    Stdframe = np.ones(Maskdata.shape)
    framemask = np.ones(Maskdata.shape)
    framemask[:,:,:] = False
    Index = []
    for i in range(Stdframe.shape[0]+1):
        if i < Stdframe.shape[0]:
            if np.nanstd(Maskdata[i,:,:]) > np.nanmedian(stddist)+2*np.nanstd(stddist): # Conditioning on std for noise
                Stdframe[i,:,:] = Maskdata[i,:,:]/(np.nanmedian(Maskdata[i,:,:])+2*np.nanstd(Maskdata[i,:,:]))
                framemask[i,:,:] = Stdframe[i,:,:]*MotionControl >= 1
                if framemask[i].any():
                    Index.append(i)
            else:
                framemask[i,:,:] = 0
    
    MaskIndex = []
    Previous = []
    AllMaskIndex = []
    i = 0
    while i < len(Index)-1:
        if Index[i+1]-Index[i] < 3:
            MaskIndex.append(i) 
        else: 
            MaskIndex = []
        if len(Previous) > 4 and len(MaskIndex) == 0:
            AllMaskIndex.append(Previous)
        Previous = MaskIndex
        i += 1
    
    if len(AllMaskIndex) > 0:
        for i in range(len(AllMaskIndex)):
            TranMask = np.ones(Maskdata[Framemin].shape)
            
            for j in range(len(AllMaskIndex[i])):
                TranMask = convolve2d(TranMask, kernel, mode="same")*framemask[AllMaskIndex[i][j]]

            if TranMask.any():
                # Generate a light curve from the transient masks
                LC = np.nansum(Maskdata*TranMask, axis = (1,2))
                LCbkg = np.nansum(Maskdata*(1-TranMask), axis = (1,2))/np.nansum((1-TranMask*Maskv2))
                fig = plt.figure()#figsize=(16,6))
                f, (ax0, ax1) = plt.subplots(1,2, gridspec_kw = {'width_ratios':[3, 1]})
                ax0.plot(time, LC-np.nansum(TranMask)*LCbkg,'.', alpha = 0.3)
                ax0.axvspan(time[AllMaskIndex[i][0]],time[AllMaskIndex[i][-1]], color = 'orange')
                ax1.imshow(TranMask, origin='lower')
                plt.title('Transient Mask')
                ax0.set_title(pixelfile.split('/')[-1].split('-')[0])
                ax0.set_xlabel('Time')
                ax0.set_ylabel('Flux')
                plt.tight_layout()
                plt.savefig(save+pixelfile.split('/')[-1].split('-')[0]+' '+str(i)+'.pdf', bbox_inches = 'tight');
                Result = 'Possible'
            else:
                Result = 'No'
    else:
        Result = 'No'

    return print(Result)

In [7]:
path = '/Volumes/TOSHIBA EXt/K2/c5/200000000/*/'
save = '/Users/ryanr/Documents/PhD/coding/Kepler/K2/c5/K2tranPix/'

for filename in glob(path+'*.gz'):
    K2tranPix(filename,save)

/Volumes/TOSHIBA EXt/K2/c5/200000000/08000/ktwo200008644-c05_lpd-targ.fits.gz
/Volumes/TOSHIBA EXt/K2/c5/200000000/08000/ktwo200008645-c05_lpd-targ.fits.gz
/Volumes/TOSHIBA EXt/K2/c5/200000000/08000/ktwo200008646-c05_lpd-targ.fits.gz
/Volumes/TOSHIBA EXt/K2/c5/200000000/08000/ktwo200008647-c05_lpd-targ.fits.gz
/Volumes/TOSHIBA EXt/K2/c5/200000000/08000/ktwo200008648-c05_lpd-targ.fits.gz
/Volumes/TOSHIBA EXt/K2/c5/200000000/08000/ktwo200008649-c05_lpd-targ.fits.gz
/Volumes/TOSHIBA EXt/K2/c5/200000000/08000/ktwo200008650-c05_lpd-targ.fits.gz
/Volumes/TOSHIBA EXt/K2/c5/200000000/08000/ktwo200008651-c05_lpd-targ.fits.gz
/Volumes/TOSHIBA EXt/K2/c5/200000000/08000/ktwo200008652-c05_lpd-targ.fits.gz
/Volumes/TOSHIBA EXt/K2/c5/200000000/08000/ktwo200008653-c05_lpd-targ.fits.gz
/Volumes/TOSHIBA EXt/K2/c5/200000000/08000/ktwo200008654-c05_lpd-targ.fits.gz
/Volumes/TOSHIBA EXt/K2/c5/200000000/08000/ktwo200008655-c05_lpd-targ.fits.gz
/Volumes/TOSHIBA EXt/K2/c5/200000000/08000/ktwo200008656-c05_lpd

/Volumes/TOSHIBA EXt/K2/c5/200000000/08000/ktwo200008719-c05_lpd-targ.fits.gz
/Volumes/TOSHIBA EXt/K2/c5/200000000/08000/ktwo200008720-c05_lpd-targ.fits.gz
/Volumes/TOSHIBA EXt/K2/c5/200000000/08000/ktwo200008721-c05_lpd-targ.fits.gz
/Volumes/TOSHIBA EXt/K2/c5/200000000/08000/ktwo200008722-c05_lpd-targ.fits.gz
/Volumes/TOSHIBA EXt/K2/c5/200000000/08000/ktwo200008723-c05_lpd-targ.fits.gz
/Volumes/TOSHIBA EXt/K2/c5/200000000/08000/ktwo200008724-c05_lpd-targ.fits.gz
/Volumes/TOSHIBA EXt/K2/c5/200000000/08000/ktwo200008725-c05_lpd-targ.fits.gz
/Volumes/TOSHIBA EXt/K2/c5/200000000/08000/ktwo200008726-c05_lpd-targ.fits.gz
/Volumes/TOSHIBA EXt/K2/c5/200000000/08000/ktwo200008727-c05_lpd-targ.fits.gz
/Volumes/TOSHIBA EXt/K2/c5/200000000/08000/ktwo200008728-c05_lpd-targ.fits.gz
/Volumes/TOSHIBA EXt/K2/c5/200000000/08000/ktwo200008729-c05_lpd-targ.fits.gz
/Volumes/TOSHIBA EXt/K2/c5/200000000/08000/ktwo200008730-c05_lpd-targ.fits.gz
/Volumes/TOSHIBA EXt/K2/c5/200000000/08000/ktwo200008731-c05_lpd

/Volumes/TOSHIBA EXt/K2/c5/200000000/08000/ktwo200008846-c05_lpd-targ.fits.gz
/Volumes/TOSHIBA EXt/K2/c5/200000000/08000/ktwo200008847-c05_lpd-targ.fits.gz
/Volumes/TOSHIBA EXt/K2/c5/200000000/08000/ktwo200008848-c05_lpd-targ.fits.gz
/Volumes/TOSHIBA EXt/K2/c5/200000000/08000/ktwo200008849-c05_lpd-targ.fits.gz
/Volumes/TOSHIBA EXt/K2/c5/200000000/08000/ktwo200008850-c05_lpd-targ.fits.gz
/Volumes/TOSHIBA EXt/K2/c5/200000000/08000/ktwo200008851-c05_lpd-targ.fits.gz
/Volumes/TOSHIBA EXt/K2/c5/200000000/08000/ktwo200008853-c05_lpd-targ.fits.gz
/Volumes/TOSHIBA EXt/K2/c5/200000000/08000/ktwo200008854-c05_lpd-targ.fits.gz
/Volumes/TOSHIBA EXt/K2/c5/200000000/08000/ktwo200008855-c05_lpd-targ.fits.gz
/Volumes/TOSHIBA EXt/K2/c5/200000000/08000/ktwo200008856-c05_lpd-targ.fits.gz
/Volumes/TOSHIBA EXt/K2/c5/200000000/08000/ktwo200008857-c05_lpd-targ.fits.gz
/Volumes/TOSHIBA EXt/K2/c5/200000000/08000/ktwo200008858-c05_lpd-targ.fits.gz
/Volumes/TOSHIBA EXt/K2/c5/200000000/08000/ktwo200008859-c05_lpd

/Volumes/TOSHIBA EXt/K2/c5/200000000/08000/ktwo200008960-c05_lpd-targ.fits.gz
/Volumes/TOSHIBA EXt/K2/c5/200000000/08000/ktwo200008961-c05_lpd-targ.fits.gz
/Volumes/TOSHIBA EXt/K2/c5/200000000/08000/ktwo200008962-c05_lpd-targ.fits.gz
/Volumes/TOSHIBA EXt/K2/c5/200000000/08000/ktwo200008963-c05_lpd-targ.fits.gz
/Volumes/TOSHIBA EXt/K2/c5/200000000/08000/ktwo200008965-c05_lpd-targ.fits.gz
/Volumes/TOSHIBA EXt/K2/c5/200000000/08000/ktwo200008966-c05_lpd-targ.fits.gz
/Volumes/TOSHIBA EXt/K2/c5/200000000/08000/ktwo200008967-c05_lpd-targ.fits.gz
/Volumes/TOSHIBA EXt/K2/c5/200000000/08000/ktwo200008968-c05_lpd-targ.fits.gz
/Volumes/TOSHIBA EXt/K2/c5/200000000/08000/ktwo200008969-c05_lpd-targ.fits.gz
/Volumes/TOSHIBA EXt/K2/c5/200000000/08000/ktwo200008970-c05_lpd-targ.fits.gz
/Volumes/TOSHIBA EXt/K2/c5/200000000/08000/ktwo200008971-c05_lpd-targ.fits.gz
/Volumes/TOSHIBA EXt/K2/c5/200000000/08000/ktwo200008972-c05_lpd-targ.fits.gz
/Volumes/TOSHIBA EXt/K2/c5/200000000/08000/ktwo200008973-c05_lpd

/Volumes/TOSHIBA EXt/K2/c5/200000000/09000/ktwo200009222-c05_lpd-targ.fits.gz
/Volumes/TOSHIBA EXt/K2/c5/200000000/09000/ktwo200009236-c05_lpd-targ.fits.gz
/Volumes/TOSHIBA EXt/K2/c5/200000000/09000/ktwo200009250-c05_lpd-targ.fits.gz
/Volumes/TOSHIBA EXt/K2/c5/200000000/09000/ktwo200009264-c05_lpd-targ.fits.gz
/Volumes/TOSHIBA EXt/K2/c5/200000000/09000/ktwo200009097-c05_lpd-targ.fits.gz
/Volumes/TOSHIBA EXt/K2/c5/200000000/09000/ktwo200009098-c05_lpd-targ.fits.gz
/Volumes/TOSHIBA EXt/K2/c5/200000000/09000/ktwo200009099-c05_lpd-targ.fits.gz
/Volumes/TOSHIBA EXt/K2/c5/200000000/09000/ktwo200009100-c05_lpd-targ.fits.gz
/Volumes/TOSHIBA EXt/K2/c5/200000000/09000/ktwo200009101-c05_lpd-targ.fits.gz
/Volumes/TOSHIBA EXt/K2/c5/200000000/09000/ktwo200009102-c05_lpd-targ.fits.gz
/Volumes/TOSHIBA EXt/K2/c5/200000000/09000/ktwo200009103-c05_lpd-targ.fits.gz
/Volumes/TOSHIBA EXt/K2/c5/200000000/09000/ktwo200009104-c05_lpd-targ.fits.gz
/Volumes/TOSHIBA EXt/K2/c5/200000000/09000/ktwo200009105-c05_lpd

/Volumes/TOSHIBA EXt/K2/c5/200000000/09000/ktwo200009167-c05_lpd-targ.fits.gz
/Volumes/TOSHIBA EXt/K2/c5/200000000/09000/ktwo200009168-c05_lpd-targ.fits.gz
/Volumes/TOSHIBA EXt/K2/c5/200000000/09000/ktwo200009169-c05_lpd-targ.fits.gz
/Volumes/TOSHIBA EXt/K2/c5/200000000/09000/ktwo200009170-c05_lpd-targ.fits.gz
/Volumes/TOSHIBA EXt/K2/c5/200000000/09000/ktwo200009171-c05_lpd-targ.fits.gz
/Volumes/TOSHIBA EXt/K2/c5/200000000/09000/ktwo200009172-c05_lpd-targ.fits.gz
/Volumes/TOSHIBA EXt/K2/c5/200000000/09000/ktwo200009173-c05_lpd-targ.fits.gz
/Volumes/TOSHIBA EXt/K2/c5/200000000/09000/ktwo200009174-c05_lpd-targ.fits.gz
/Volumes/TOSHIBA EXt/K2/c5/200000000/09000/ktwo200009175-c05_lpd-targ.fits.gz
/Volumes/TOSHIBA EXt/K2/c5/200000000/09000/ktwo200009176-c05_lpd-targ.fits.gz
/Volumes/TOSHIBA EXt/K2/c5/200000000/09000/ktwo200009177-c05_lpd-targ.fits.gz
/Volumes/TOSHIBA EXt/K2/c5/200000000/09000/ktwo200009178-c05_lpd-targ.fits.gz
/Volumes/TOSHIBA EXt/K2/c5/200000000/09000/ktwo200009179-c05_lpd

/Volumes/TOSHIBA EXt/K2/c5/200000000/09000/ktwo200009280-c05_lpd-targ.fits.gz


## Revamped version

In [6]:
def K2tranPix(pixelfile,save): # More efficient in checking frames
    hdu = fits.open(pixelfile)
    dat = hdu[1].data
    datacube = fits.ImageHDU(hdu[1].data.field('FLUX')[:]).data
    time = dat["TIME"] + 2454833.0

    # Finding the reference frame
    n_steps = 12
    std_vec = np.zeros(n_steps)
    for i in range(n_steps):
        std_vec[i] = np.nanstd(datacube[i:-n_steps+i:n_steps,:,:] - datacube[i+n_steps*80,:,:])
    Framemin = np.where(std_vec==np.nanmin(std_vec))[0][0]

    # Remove all negative flux values, not physical
    #datacube[datacube < 0] = 0

    # Make a mask of the target object, using the reference frame 
    Mask = datacube[Framemin,:,:]/(np.nanmedian(datacube[Framemin,:,:])+np.nanstd(datacube[Framemin,:,:]))
    Mask[Mask>=1] = np.nan
    Mask[Mask<1] = 1
    # Generate a second mask from remainder of the first. This grabs the fainter pixels around known sources
    Maskv2 = datacube[Framemin,:,:]*Mask/(np.nanmedian(datacube[Framemin,:,:]*Mask)+np.nanstd(datacube[Framemin,:,:]*Mask))
    Maskv2[Maskv2>=1] = np.nan
    Maskv2[Maskv2<1] = 1
    # Apply mask to data
    Maskdata = datacube*Maskv2
    #Maskdata[Maskdata < 0] = 0
    #Maskdata = Maskdata[Framemin::12,:,:]
    # Calculating the standard deviations of all frames, from which the significance of each
    # frame's std can be compared.
    stddist = np.zeros(Maskdata.shape[0])
    for i in range(Maskdata.shape[0]):
        stddist[i] = np.nanstd(Maskdata[i,:,:])
    
    kernel = np.ones((3,3))
    temp = np.ma.masked_invalid(Maskv2)
    Conv = convolve2d(temp.mask, kernel, mode="same")
    MotionCotrol = Conv < 2

    Stdframe = np.ones(Maskdata.shape)
    framemask = np.ones(Maskdata.shape)

    Index = []
    for i in range(Stdframe.shape[0]+1):
        if i < Stdframe.shape[0]:
            framemask[:,:,:] = False
            if np.nanstd(Maskdata[i,:,:]) > np.nanmedian(stddist)+2*np.nanstd(stddist): # Conditioning on std for noise
                Stdframe[i,:,:] = Maskdata[i,:,:]/(np.nanmedian(Maskdata[i,:,:])+2*np.nanstd(Maskdata[i,:,:]))
                if Maskdata.shape[1]>1 and Maskdata.shape[2]>1:
                    framemask[i,:,:] = Stdframe[i,:,:]*MotionCotrol <= 1
                else:
                    framemask[i,:,:] = Stdframe[i,:,:]*(Conv<1) < 1
                if framemask[i].any():
                    Index.append(i)
            else:
                framemask[i,:,:] = 0



    def pix2coord(x,y):
        wx, wy = mywcs.wcs_pix2world(x, y, 0)
        return np.array([float(wx), float(wy)])

    MaskIndex = []
    Previous = []
    AllMaskIndex = []
    i = 0
    while i < len(Index)-1:
        if Index[i+1]-Index[i] < 2:
            MaskIndex.append(i) 
        else: 
            MaskIndex = []
        if len(Previous) > 3 and len(MaskIndex) == 0:
            AllMaskIndex.append(Previous)
        Previous = MaskIndex
        i += 1
    thingo = []
    if len(AllMaskIndex) > 0:
        for i in range(len(AllMaskIndex)):
            TranMask = np.ones(Maskdata[Framemin].shape)

            for j in range(len(AllMaskIndex[i])):
                TranMask = convolve2d(TranMask, kernel, mode="same")*framemask[AllMaskIndex[i][j]] # Gives frames room for motion
            TranMask = convolve2d(TranMask, kernel, mode="same")
            if TranMask.any():
                # Generate a light curve from the transient masks
                LC = np.nansum(datacube*TranMask, axis = (1,2))
                #Find Coords of transient
                funny_keywords = {'1CTYP4': 'CTYPE1',
                      '2CTYP4': 'CTYPE2',
                      '1CRPX4': 'CRPIX1',
                      '2CRPX4': 'CRPIX2',
                      '1CRVL4': 'CRVAL1',
                      '2CRVL4': 'CRVAL2',
                      '1CUNI4': 'CUNIT1',
                      '2CUNI4': 'CUNIT2',
                      '1CDLT4': 'CDELT1',
                      '2CDLT4': 'CDELT2',
                      '11PC4': 'PC1_1',
                      '12PC4': 'PC1_2',
                      '21PC4': 'PC2_1',
                      '22PC4': 'PC2_2'}
                mywcs = {}
                for oldkey, newkey in funny_keywords.items():
                    mywcs[newkey] = hdu[1].header[oldkey] 
                mywcs = WCS(mywcs)
                x, y = np.where(TranMask == 1)
                Coord = pix2coord(x[0],y[0])

                fig = plt.figure()
                # set up subplot grid
                gridspec.GridSpec(3,3)

                # large subplot
                plt.subplot2grid((3,3), (0,0), colspan=2, rowspan=3)
                plt.title('Masked light curve (JD '+str(time[AllMaskIndex[i][0]])+'RA '+str(Coord[0])+' DEC '+str(Coord[1]))
                plt.xlabel('Time (BJD)')
                plt.ylabel('Flux')
                plt.plot(time, LC,'.')
                plt.axvspan(time[AllMaskIndex[i][0]],time[AllMaskIndex[i][-1]], color = 'orange')
                plt.xlim(time[AllMaskIndex[i][0]]-5,time[AllMaskIndex[i][-1]]+5)


                # small subplot 1
                plt.subplot2grid((3,3), (0,2))
                plt.title('Reference')
                #plt.xlabel('Data values')
                #plt.ylabel('Frequency')
                plt.imshow(Maskdata[Framemin,:,:], origin='lower',vmax=200)

                # small subplot 2
                plt.subplot2grid((3,3), (1,2))

                plt.title('Mask')
                #plt.xlabel('Data values')
                #plt.ylabel('Frequency')
                plt.imshow(TranMask, origin='lower')

                # small subplot 3
                plt.subplot2grid((3,3), (2,2))
                plt.title('Event')
                #plt.xlabel('Data values')
                #plt.ylabel('Frequency')
                plt.imshow(Maskdata[AllMaskIndex[i][1],:,:], origin='lower',vmax=100)

                # fit subplots and save fig
                fig.tight_layout()
                #fig.set_size_inches(w=11,h=7)
                plt.savefig(save+pixelfile.split('/')[-1].split('-')[0]+' '+str(i)+'.pdf', bbox_inches = 'tight');
                Result = 'Possible'
                print('Possible')
                thingo.append(TranMask)
            else:
                Result = 'No'
    else:
        Result = 'No'

    return print(pixelfile)

In [ ]:
tes = []
tes.append(test)

In [ ]:
tes.append(test2)

In [ ]:
tes[0]

In [ ]:
plt.imshow(datacube[Framemin,:,:]/(np.nanmedian(datacube[Framemin,:,:])+2*np.nanstd(datacube[Framemin,:,:])))
plt.colorbar();

In [ ]:
plt.imshow(Maskv2)
plt.show()
plt.imshow(Negmask[Framemin])
plt.show();

In [ ]:
def plttpf(f):
    plt.imshow(Stdframe[f,:,:],vmin = 0, vmax=1)
    
    plt.colorbar()
    plt.show()

In [ ]:
interact(plttpf, f=(0,Stdframe.shape[0],1));

In [ ]:
test = datacube[Framemin]>0
plt.imshow(test*datacube[Framemin])
plt.colorbar()

In [ ]:
test = datacube[Framemin]>0
plt.imshow(test)

In [ ]:
pixelfile = '/Volumes/TOSHIBA EXT/K2/c5/200000000/08000/ktwo200008644-c05_lpd-targ.fits.gz'
#pixelfile = '../data/ktwo212535880-c06_lpd-targ.fits'
save = '/Users/ryanr/Documents/PhD/coding/Kepler/K2/c5/'

In [ ]:

# Loading in data
hdu = fits.open(pixelfile)
dat = hdu[1].data
datacube = dat["FLUX"]
time = dat["TIME"]

# Finding the reference frame
n_steps = 12
std_vec = np.zeros(n_steps)
for i in range(n_steps):
    std_vec[i] = np.nanstd(datacube[i:-n_steps+i:n_steps,:,:] - datacube[i+n_steps*80,:,:])
Framemin = np.where(std_vec==np.nanmin(std_vec))[0][0]

# Remove all negative flux values, not physical
datacube[datacube < 0] = np.nan

# Make a mask of the target object, using the reference frame 
Mask = datacube[Framemin,:,:]/(np.nanmedian(datacube[Framemin,:,:])+2*np.nanstd(datacube[Framemin,:,:]))
Mask[Mask>=1] = np.nan
Mask[Mask<1] = 1
# Generate a second mask from remainder of the first. This grabs the fainter pixels around known sources
Maskv2 = datacube[Framemin,:,:]*Mask/(np.nanmedian(datacube[Framemin,:,:]*Mask)+2*np.nanstd(datacube[Framemin,:,:]*Mask))
Maskv2[Maskv2>=1] = np.nan
Maskv2[Maskv2<1] = 1
# Apply mask to data
Maskdata = datacube*Maskv2
Maskdata[Maskdata < 0] = np.nan
#Maskdata = Maskdata[Framemin::12,:,:]
# Calculating the standard deviations of all frames, from which the significance of each
# frame's std can be compared.
stddist = np.zeros(Maskdata.shape[0])
for i in range(Maskdata.shape[0]):
    stddist[i] = np.nanstd(Maskdata[i,:,:])

# Section to calculate if any points are significant in a frame
Stdframe = np.ones(Maskdata.shape)
framemask = np.ones(Maskdata.shape)
framemask[:,:,:] = False
Index = []
for i in range(Stdframe.shape[0]+1):
    if i < Stdframe.shape[0]:
        if np.nanstd(Maskdata[i,:,:]) > np.nanmedian(stddist)+2*np.nanstd(stddist): # Conditioning on std for noise
            Stdframe[i,:,:] = Maskdata[i,:,:]/(np.nanmedian(Maskdata[i,:,:])+2*np.nanstd(Maskdata[i,:,:]))
            framemask[i,:,:] = Stdframe[i,:,:] >= 1
            Index.append(i)
        else:
            framemask[i,:,:] = 0

MaskIndex = []
Previous = []
AllMaskIndex = []
i = 0
while i < len(Index)-1:
    if Index[i+1]-Index[i] < 3:
        MaskIndex.append(i) 
    else: 
        MaskIndex = []
    if len(Previous) > 3 and len(MaskIndex) == 0:
        AllMaskIndex.append(Previous)
    Previous = MaskIndex
    i += 1

if len(AllMaskIndex) > 0:
    for i in range(len(AllMaskIndex)):
        TranMask = np.ones(Maskdata[Framemin].shape)
        #print(len(MaskIndex))
        for j in range(len(AllMaskIndex[i])):
            TranMask = TranMask*framemask[AllMaskIndex[i][j]]

        if TranMask.any():
            # Generate a light curve from the transient masks
            LC = np.nansum(datacube*TranMask, axis = (1,2))
            fig = plt.figure()#figsize=(16,6))
            f, (ax0, ax1) = plt.subplots(1,2, gridspec_kw = {'width_ratios':[3, 1]})
            ax0.plot(time, LC,'.')

            ax1.imshow(TranMask, origin='lower')
            plt.title('Transient Mask')
            ax0.set_title(pixelfile.split('/')[-1].split('-')[0])
            ax0.set_xlabel('Time')
            ax0.set_ylabel('Flux')
            plt.tight_layout()
            plt.savefig(save+pixelfile.split('/')[-1].split('-')[0]+' '+str(i)+'.pdf', bbox_inches = 'tight');
            Result = 'Possible'
        else:
            Result = 'No'
else:
    Result = 'No'



In [ ]:
AllMaskIndex[0][1]

In [ ]:
MaskIndex = []
while i < len(Index)-1 and len(MaskIndex) < 3:
    if Index[i+1]-Index[i] < 3:
        MaskIndex.append(i) 
    else: 
        MaskIndex = []
    i += 1

if len(MaskIndex) > 0:
    TranMask = np.ones(Maskdata[Framemin].shape)
    for i in range(len(MaskIndex)):
        TranMask = TranMask*framemask[MaskIndex[i]]
        
    if TranMask.any():
        # Generate a light curve from the transient masks
        LC = np.nansum(datacube*TranMask, axis = (1,2))
        fig = plt.figure()#figsize=(16,6))
        f, (ax0, ax1) = plt.subplots(1,2, gridspec_kw = {'width_ratios':[3, 1]})
        ax0.plot(time, LC,'.')

        ax1.imshow(tranmask[j], origin='lower')
        plt.title('Transient Mask')
        ax0.set_title(pixelfile.split('/')[-1].split('-')[0])
        ax0.set_xlabel('Time')
        ax0.set_ylabel('Flux')
        plt.tight_layout()
        plt.savefig(save+pixelfile.split('/')[-1].split('-')[0]+' '+str(j)+'.pdf', bbox_inches = 'tight');
        Result = 'Possible'
else:
    Result = 'No'
   

In [ ]:
pixelfile = '/Volumes/TOSHIBA EXT/K2/c5/200000000/08000/ktwo200008645-c05_lpd-targ.fits.gz'
#pixelfile = '../data/ktwo212535880-c06_lpd-targ.fits'
save = '/Users/ryanr/Documents/PhD/coding/Kepler/K2/c5/'

In [ ]:
MaskIndex = []
i = 0
while i < len(Index)-1 and len(MaskIndex) < 3:
    if Index[i+1]-Index[i] < 2:
        MaskIndex.append(i) 
        print(MaskIndex)
    else: 
        MaskIndex = []
    i += 1

In [ ]:
MaskIndex

# Checking the distribution of standard deviations 

With the initial method of flagging events that were 2$\sigma$ or more over the background picked up a lot of noise. This occured in frames that had a low $\sigma$, thus small noise fluxuations would meet the requirement. A workaround for this is to add the following threashold on the analysis of each frame,
        $$\rm \sigma_{frame}>Median(\sigma_{total}) + std(\sigma_{total}) $$
this criteria demands that only frames with a $\sigma$ greater than the median of all frame's standard deviations plus the standard deviation of all standard deviations are concidered for analysis. This additional criteria is justifiable as events that feature a transient will skew the frames $\sigma$ to a value higher than that produced by noise. As seen in the plot below this criteria cuts most frames from the analysis.

In [ ]:
stddist = np.zeros(Maskdata.shape[0])
for i in range(Maskdata.shape[0]):
    stddist[i] = np.nanstd(Maskdata[i,:,:])
plt.figure()
plt.hist(stddist[~np.isnan(stddist)], bins = 'auto')
plt.axvline(np.nanmedian(stddist)+np.nanstd(stddist), color = 'k')
plt.xlim(0,10)
plt.ylabel('Number of frames')
plt.xlabel('Standard deviation of frame');

In [ ]:
np.nanstd(stdd)

In [ ]:
2*np.nanstd(datacube[Framemin]*Maskv2)

In [ ]:
t = np.array(1)

In [ ]:
t.any()

In [10]:
np.nansum(Maskdata*(1-TranMask))

NameError: name 'Maskdata' is not defined

In [1]:
data

NameError: name 'data' is not defined